# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manay,7.2150,126.5397,302.22,65,99,4.14,PH,2024-09-17
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,287.48,38,20,5.00,RU,2024-09-17
2,2,puerto ayora,-0.7393,-90.3518,294.57,95,62,1.79,EC,2024-09-17
3,3,hawaiian paradise park,19.5933,-154.9731,301.89,77,100,3.09,US,2024-09-17
4,4,salinopolis,-0.6136,-47.3561,300.00,79,1,6.03,BR,2024-09-17


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.7,
    color = "City",
    alpha = 0.4
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Humidity'] > 85) & (city_data_df['Humidity'] < 110)]

# Drop any rows with null values
city_data_df = city_data_df[(city_data_df['Humidity'] > 85) & (city_data_df['Humidity'] < 110)].dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,puerto ayora,-0.7393,-90.3518,294.57,95,62,1.79,EC,2024-09-17
5,5,kachia,9.8667,7.9500,294.37,93,100,0.84,NG,2024-09-17
11,11,colonia,50.9333,6.9500,288.66,87,20,3.09,DE,2024-09-17
14,14,bredasdorp,-34.5322,20.0403,281.12,89,9,1.08,ZA,2024-09-17
17,17,port-aux-francais,-49.3500,70.2167,277.55,96,100,15.76,TF,2024-09-17


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lng,Humidity,Hotel Name
2,puerto ayora,EC,-0.7393,-90.3518,95,
5,kachia,NG,9.8667,7.9500,93,
11,colonia,DE,50.9333,6.9500,87,
14,bredasdorp,ZA,-34.5322,20.0403,89,
17,port-aux-francais,TF,-49.3500,70.2167,96,
18,adamstown,PN,-25.0660,-130.1015,87,
20,coquimbo,CL,-29.9533,-71.3436,87,
27,aasiaat,GL,68.7098,-52.8699,89,
29,waitangi,NZ,-43.9535,-176.5597,92,
30,tiksi,RU,71.6872,128.8694,96,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": filter,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kachia - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
bredasdorp - nearest hotel: Victoria Hotel
port-aux-francais - nearest hotel: Keravel
adamstown - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
aasiaat - nearest hotel: SØMA
waitangi - nearest hotel: Hotel Chathams
tiksi - nearest hotel: Арктика
patong - nearest hotel: Thara Patong Beach Resort & Spa
stanley - nearest hotel: Hotel 52
isafjordur - nearest hotel: Hótel Horn
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
fort bragg - nearest hotel: Airborne Inn Lodging
ribeira grande - nearest hotel: Hospedaria JSF
iskateley - nearest hotel: Авантаж
ijebu-igbo - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
chonchi - nearest hotel: Hotel Huildín
anadyr - nearest hotel: Гостевой дом
padang - nearest hotel: losmen Surya
kangasala - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
2,puerto ayora,EC,-0.7393,-90.3518,95,Hostal La Mirada De Solitario George
5,kachia,NG,9.8667,7.9500,93,No hotel found
11,colonia,DE,50.9333,6.9500,87,Wasserturm Hotel Cologne
14,bredasdorp,ZA,-34.5322,20.0403,89,Victoria Hotel
17,port-aux-francais,TF,-49.3500,70.2167,96,Keravel
...,...,...,...,...,...,...
550,banyo,CM,6.7500,11.8167,92,Hôtel Amin
551,meulaboh,ID,4.1363,96.1285,93,Hotel Meuligou
553,tamiahua,MX,21.2667,-97.4500,87,No hotel found
556,nouadhibou,MR,20.9310,-17.0347,88,Hotel valencia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.7,
    color = "City",
    alpha = 0.4,
    hover_cols = ["Lng", "Lat", "City", "Humidity","Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)